In [88]:
import pandas as pd
import chardet


with open("D:/05.japanproject/jp/freq/spoken_freq_nlb.txt", "rb") as f:
    result = chardet.detect(f.read())
    print(f"检测到编码: {result['encoding']}")

spoken_freq_nlb_txt_path = "D:/05.japanproject/jp/freq/spoken_freq_nlb.txt"
spoken_freq_nlb_csv_path = "D:/05.japanproject/jp/freq/spoken_freq_nlb.csv"
spoken_freq_nlb_sumed_csv_path = "D:/05.japanproject/jp/freq/spoken_freq_nlb_sumed.csv"

df = pd.read_csv(
    spoken_freq_nlb_txt_path,
    header=None,
    encoding="utf-8-sig",
    sep="\t",
    on_bad_lines="skip",
    quotechar='"',
    engine="python",
)

df.drop([0, 7], axis=1, inplace=True)
df.columns = ["headword", "type", "reading", "freq", "freq_per", "rank"]

检测到编码: UTF-8-SIG


In [89]:
df.describe()

,freq_per,rank
count,98177.000000,98176.000000
mean,0.101427,0.499986
std,3.585749,0.288672
min,0.000000,0.000000
25%,0.000100,0.249988
50%,0.001000,0.499975
75%,0.008900,0.749982
max,489.627200,0.999990


In [90]:
def get_unicode_codepoints(text):
    """获取日语文本中每个字符的Unicode码点（格式为U+XXXX）"""
    return "".join([f"{ord(char):04X}" for char in text])


japanese_word = "こんにちは"
get_unicode_codepoints(japanese_word)

'30533093306B3061306F'

In [91]:
df.dropna(subset=["headword", "type", "reading", "freq", "rank"], inplace=True)
df["freq"] = pd.to_numeric(df["freq"].str.replace(",", ""), errors="coerce")

# 2. 将次数映射到新列
value_counts = df["headword"].value_counts()
df["count"] = df["headword"].map(value_counts)

mask = (df["type"] == "記号") | (df["headword"] == "*")
df = df[~mask].copy()
df.sort_values(by="freq", ascending=False, inplace=True)
print("ok")

ok


In [92]:
df["hrt"] = df["headword"] + "-" + df["reading"] + "-" + df["type"]
hrt_counts = df["hrt"].value_counts()
df["hrt_count"] = df["hrt"].map(hrt_counts)
df["r"] = (
    df.groupby("hrt")["rank"]
    .rank(
        method="first", ascending=False
    )  # 并列时按出现顺序排名（类似 SQL 的 ROW_NUMBER）  # 升序排列（默认）
    .astype(int)
)
print("ok")

ok


In [93]:
df[df["r"] == 1].to_csv(spoken_freq_nlb_csv_path, index=False, encoding="utf-8-sig")
# df.to_csv(spoken_freq_nlb_csv_path, index=False, encoding="utf-8-sig")
# df.to_csv(spoken_freq_nlb_csv_path, index=False, encoding="utf-8-sig")
print("ok")

ok


In [97]:
spoken_freq_nlb_sumed_csv_path = "D:/05.japanproject/jp/freq/spoken_freq_nlb_sumed.csv"
a = df["freq"].groupby(df["headword"]).sum().to_frame()
a.rename(columns={"freq": "spoken_freq"}, inplace=True)

a.sort_values(by="spoken_freq", ascending=False, inplace=True)
a.to_csv(spoken_freq_nlb_sumed_csv_path, encoding="utf-8-sig")
print("ok")

           freq
headword       
7する          23
8         12054
8する           1
Tシャツ       4083
Δ変調           2
...         ...
ｓｕｐｅｒ         3
ｔ          1760
ｔｈｅ          60
ｖ           119
ｗ            20

[96089 rows x 1 columns]
ok


In [8]:
def get_unicode_codepoints(text):
    """将文本中每个字符转换为4位十六进制Unicode码点字符串（无U+前缀）"""
    return "".join([f"{ord(char):04X}" for char in text])


# 创建示例 DataFrame
data = {
    "text_column": ["日本語", "こんにちは", "東京", "日本語"],  # 包含重复文本
    "value": [100, 200, 300, 400],
}
df = pd.DataFrame(data)

# 设置索引（处理重复值）
df.index = (
    df["text_column"].apply(get_unicode_codepoints)
    + "_"
    + df.groupby("text_column").cumcount().astype(str)
)

# 输出结果
print(df)

                       text_column  value
65E5672C8A9E_0                 日本語    100
30533093306B3061306F_0       こんにちは    200
67714EAC_0                      東京    300
65E5672C8A9E_1                 日本語    400
